In [ ]:
from pickle import load, dump
from rotor_mtm.results import IntegrationResults
from os import listdir, getcwd
from os.path import isfile, join
import plotly.graph_objs as go

def get_results_chain(f,
                slope=False):

    sl_str = ''
    if slope:    
        sl_str = ' slope'
    
    try:
        with open(f'results_NL_Chain{sl_str}_f_{f}.pic', 'rb') as file:
            res = load(file)
        res = IntegrationResults.update_class_object(res)
        
    except:
        f = float(f)
        with open(f'results_NL_Chain{sl_str}_f_{f}.pic', 'rb') as file:
            res = load(file)
        res = IntegrationResults.update_class_object(res)
    
           
    return res

def plot_frf_set_chain(slope=False,
                        dof=[19],
                        plot_linear=True,
                        plot_rigid=True,
                        remove_list=[],
                        amplitude_units='rms'):
    

    if type(dof) != list:
        dof = [dof]

    sl_str = ''
    if slope:    
        sl_str = ' slope'
              
    filename_base = f'results_NL_Chain{sl_str}'

    picfiles = [f for f in listdir(getcwd()) if (isfile(join(getcwd(), f)) and '.pic' in f and f.startswith(filename_base) and all([str(s) not in f for s in remove_list]) and f[len(filename_base)] == '_')]
    

    f0 = []
    fig = go.Figure()    

    # for i, f in enumerate(picfiles):
        
    #     try:
    #         print(float(f[len(filename_base) + 3 : -4]))
    #     except:
    #         picfiles.remove(f)
            
    print(picfiles)

    for i, f in enumerate(picfiles):

        print(f)

        f0.append(float(f[len(filename_base) + 3 : -4]))
        print(f0[-1])

        with open(f, 'rb') as file:
            res = load(file)
        res = IntegrationResults.update_class_object(res)

        force_units = 'N'
        force_units_legend = force_units
        amp_units = f'm ({amplitude_units}) / ' + force_units
        corr1 = 1
        corr2 = f0[-1]
        
        fig_aux = res.plot_frf(dof=dof,
                                amplitude_units=amplitude_units)
        name = f'{f0[-1]} {force_units_legend}'        
        fig_aux.data[0].name = name
        fig_aux.data[0].y = fig_aux.data[0].y / corr2

        if i == 0:
            if 'linear_results' in res.__dict__:
                fig_lin = res.linear_results.plot_frf(dof=dof,
                                                    whirl='forward',
                                                    amplitude_units=amplitude_units)
                fig_lin.data[0].line.color = 'blue'
                fig_lin.data[0].y = fig_lin.data[0].y / corr1
                fig_lin.data[0].name = 'Linear Resonators'

            if 'rigid_results' in res.__dict__:
                fig_rig = res.rigid_results.plot_frf(dof=dof,
                                                    whirl='forward',
                                                    amplitude_units=amplitude_units)
                fig_rig.data[0].line.color = 'black'
                fig_rig.data[0].line.dash = 'dash'
                fig_rig.data[0].y = fig_rig.data[0].y / corr1
                fig_rig.data[0].name = 'Bare Chain'
            
            if fig_lin and plot_linear:
                plot_linear = False
                fig = fig_lin

            if fig_rig and plot_rigid:
                plot_rigid = False
                if len(fig.data) == 0:
                    fig = fig_rig
                else:
                    fig.add_trace(fig_rig.data[0])

            if len(fig.data) == 0:
                fig = fig_aux
            else:
                fig.add_trace(fig_aux.data[0])

        else:
            fig.add_trace(fig_aux.data[0])

    fig.update_layout(yaxis_title=amp_units,
                          width=900)
    
    return fig


    

In [ ]:
res = get_results_chain(f=1000., slope=True)

In [ ]:
for f in [1000., 3000., 6000., 9000.]:
    for slope in [True, False]:

        res = get_results_chain(f=f, 
                                slope=slope)

        fig1 = res.plot_spectrum_heatmap(dof=res.system.dof_nl[0] - 1,
                                        max_freq=1400,
                                        hanning=True,
                                        cut=3)#.update_layout(width=1000,
                                                            # height=600)
        

        fig2 = res.plot_diff_map(dof=[(res.system.dof_nl[i], res.system.base_dof[i]) for i in range(10)],
                                mode='angle',
                                hanning=False,
                                max_freq=1400,
                                min_freq=10,
                                cut=3,
                                log_mode=True)#.update_layout(width=1000,
                                                            # height=600)
        
        fig3 = res.plot_diff_map(dof=[(res.system.dof_nl[i], res.system.base_dof[i]) for i in range(10)],
                                mode='amp',
                                hanning=False,
                                max_freq=1400,
                                min_freq=10,
                                cut=3,
                                log_mode=True)#.update_layout(width=1000,
                                                            # height=600)
        
        fig1.write_image(f'heatmap last {f} - slope-{slope}.pdf')
        fig2.write_image(f'heatmap phase {f} - slope-{slope}.pdf')
        fig3.write_image(f'heatmap amplification {f} - slope-{slope}.pdf')
        print(f'{f} - slope-{slope}')


In [ ]:
res.plot_spectrum_heatmap(dof=res.system.dof_nl[0] - 1,
                          max_freq=1400,
                          hanning=True,
                          cut=3).update_layout(width=1000,
                                               height=600)

In [ ]:
res.plot_diff_map(dof=[(res.system.dof_nl[i], res.system.base_dof[i]) for i in range(10)],
                  mode='angle',
                  hanning=False,
                  max_freq=1400,
                  min_freq=10,
                  cut=3,
                  log_mode=True).update_layout(width=1000,
                                               height=600)

In [ ]:
f_list = [
        1000,
        2000, 
        3000, 
        4000, 
        5000, 
        6000, 
        7000, 
        8000, 
        9000
        ]

for f in [1000, 3000, 6000, 9000]:
        for slope in [False, True]:
                plot_frf_set_chain(slope=slope,
                                dof=[19],
                                plot_linear=True,
                                plot_rigid=True,
                                remove_list=[a for a in f_list if a != f],
                                amplitude_units='rms').write_image(f'graficos tese/FRF {float(f)} - slope-{slope}.pdf')


In [ ]:
res = get_results_chain(1000)
# dict_aux = {}

# for k in res.rigid_results.rf.keys():
#     dict_aux[int(k)] = res.rigid_results.rf[k]

# res.rigid_results.rf = dict_aux

res.ddl[0].keys()

In [ ]:
plot_frf_set_chain()

In [ ]:
for f in [1000., 2000., 3000., 4000., 
          5000., 6000., 7000., 8000., 9000.]:
    
    print(f)
    for slope in [True, False]:

        sl_str = ''
        if slope:    
            sl_str = ' slope'

        res = get_results_chain(f=f, 
                        slope=slope)
        lin_r = res.linear_system.calc_linear_frf(res.fl,
                                                silent=True)
        rig_r = res.rigid_system.calc_linear_frf(res.fl,
                                                silent=True)
        res.linear_results = lin_r
        res.rigid_results = rig_r
        
        filename = f'results_NL_Chain{sl_str}_f_{f}.pic'
        with open(filename, 'wb') as file:
            dump(res, file)

In [ ]:
lin_r = res.rigid_system.calc_linear_frf(res.fl,
                                  silent=True)
lin_r.plot_frf(dof=['19'],
                whirl='forward',
                amplitude_units='rms')

In [ ]:
res.rigid_system.C